In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [10]:
class WVModel(nn.Module):
    
    #loosely based on https://github.com/salesforce/awd-lstm-lm/blob/master/model.py
    def __init__(self, wordVecSize, numHiddenLayers, hiddenSize, hiddenSizes = None):
        super().__init__()
        self.getSize = lambda x: (wordVecSize if (x == 0 or x == numHiddenLayers + 2) else (hiddenSizes[x - 1] if hiddenSizes else hiddenSize))
        self.layers = nn.ModuleList([torch.nn.LSTM(self.getSize(l), self.getSize(l + 1)) for l in range(numHiddenLayers + 2)])
        
    def forward(self, input, hidden):
        
        new_hidden = []
        for layer, rnn in enumerate(self.layers):
            current_output, current_new_hidden = rnn(input, hidden[layer])
            input = current_output
            new_hidden.append(current_new_hidden)
            
        output = input
        return output, new_hidden
    
    #copied almost exactly from https://github.com/salesforce/awd-lstm-lm/blob/master/model.py
    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        return [(weight.new(1, bsz, self.getSize(l + 1)).zero_(),
                weight.new(1, bsz, self.getSize(l + 1)).zero_())
                for l in range(numHiddenLayers + 2)]

In [17]:
wordVecSize = 3
numHiddenLayers = 4
hiddenSize = 4
hiddenSizes = [1, 2, 3, 2, 1]

model = WVModel(wordVecSize, numHiddenLayers, hiddenSize, hiddenSizes)

batch_size = 1
input = Variable(torch.Tensor([[[0, 1, 0]]]))
hidden = model.init_hidden(batch_size)
output, hidden = model(input, hidden)

In [19]:
def train():
    wordVecSize = 3
    numHiddenLayers = 4
    hiddenSize = 4
    hiddenSizes = [1, 2, 3, 2, 1]

    model = WVModel(wordVecSize, numHiddenLayers, hiddenSize, hiddenSizes)


tensor([[[-0.0011,  0.0285,  0.0928]]])